<a target="_blank" href="https://colab.research.google.com/github/greatakela/GenChatBot/blob/main/Notebooks/GNLP_HW2-data_prep.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
data_path = '/content/drive/MyDrive/GNLP/HW2/'

Создание датасета на основе диалогов из фильмов STAR TREK.

In [ ]:
%%capture
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
!git clone https://github.com/varenc/star_trek_transcript_search


Cloning into 'star_trek_transcript_search'...
remote: Enumerating objects: 803, done.
remote: Counting objects: 100% (803/803), done.
remote: Compressing objects: 100% (791/791), done.
remote: Total 803 (delta 12), reused 792 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (803/803), 8.83 MiB | 19.03 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [ ]:
%cd ./star_trek_transcript_search/scripts/

/content/star_trek_transcript_search/scripts


In [ ]:
!ls

Discovery  DS9	Enterprise  Movies  NextGen  SNW  TAS  TOS  Voyager


In [ ]:
import pandas as pd
import re
import os
import random

def parse_transcript(filepath):
    data = []

    with open(filepath, 'r', encoding='utf-8') as file:
        text = file.read()

    # Correct bracket types and selectively remove text in parentheses
    text = text.replace('{', '[').replace('}', ']')
    text = re.sub(r'\([^\)]*\)', '', text)
    text = re.sub(r'\[OC\]|\[on monitor\]', '', text)
    #text = re.sub(r'\((?![^\(\)]*\[OC\])[^\(\)]*\)', '', text)
    #text = re.sub(r'\((?![^\(\)]*\[OC\])[^\(\)]*\)', '', text)

    episode_match = re.search(r'Episode Number: (\d+)', text)
    episode = int(episode_match.group(1)) if episode_match else None

    # Split the text into segments for scene names and dialogues
    segments = re.split(r'(\[[^\]]+\])', text)
    current_scene = "Unknown"

    for i in range(1, len(segments), 2):
        scene_name = segments[i].strip("[]")
        if not scene_name.endswith("OC") and not scene_name.endswith("on monitor"):
            current_scene = scene_name
        dialogues = segments[i + 1].split('\n')

        for dialogue in dialogues:
            if ':' in dialogue:
                character_dialogue = dialogue.split(':', 1)
                character = character_dialogue[0].strip()
                # Strip [OC] and [on monitor] from character names
                character = re.sub(r'\s*\[OC\]$', '', character, flags=re.IGNORECASE)
                character = re.sub(r'\s*\[on monitor\]$', '', character, flags=re.IGNORECASE)
                text = character_dialogue[1].strip()
                data.append([episode, current_scene, character, text])

    return data

# List of folders to process
folders = ["TOS", "TAS", "Movies"]  # Replace these with your actual folder paths

# Initialize an empty DataFrame to store all data
final_df = pd.DataFrame(columns=["Episode", "Scene", "Character", "Text"])

# Iterate over each folder
for folder in folders:
    current_folder_path = os.path.join(os.getcwd(), folder)  # Adjust if folders are not in the current directory
    text_files = sorted([f for f in os.listdir(current_folder_path) if f.endswith('.txt')])

    all_data = []

    # Process each file in the current folder
    for filename in text_files:
        filepath = os.path.join(current_folder_path, filename)
        if os.path.isfile(filepath):
            file_data = parse_transcript(filepath)
            all_data.extend(file_data)

    # Append data from the current folder to the final DataFrame
    final_df = pd.concat([final_df, pd.DataFrame(all_data, columns=["Episode", "Scene", "Character", "Text"])], ignore_index=True)

# Verify the output
print(final_df.head())


  Episode   Scene Character                                               Text
0       1  Bridge     SPOCK                                 Check the circuit.
1       1  Bridge     TYLER                                All operating, sir.
2       1  Bridge     SPOCK  It can't be the screen then. Definitely someth...
3       1  Bridge     TYLER                      It could be these meteorites.
4       1  Bridge       ONE  No, it's something else. There's still somethi...


In [ ]:
final_df.shape

(40285, 4)

In [ ]:
final_df[:25]

,Episode,Scene,Character,Text
0,1,Bridge,SPOCK,Check the circuit.
1,1,Bridge,TYLER,"All operating, sir."
2,1,Bridge,SPOCK,It can't be the screen then. Definitely someth...
3,1,Bridge,TYLER,It could be these meteorites.
4,1,Bridge,ONE,"No, it's something else. There's still somethi..."
5,1,Bridge,TYLER,"It's coming at the speed of light, collision c..."
6,1,Bridge,ONE,"Evasive manoeuvres, sir?"
7,1,Bridge,PIKE,Steady as we go.
8,1,Bridge,GARISON,"It's a radio wave, sir. We're passing through ..."
9,1,Bridge,PIKE,They were keyed to cause interference and attr...


In [ ]:
final_df['Scene'].nunique()

674

In [ ]:
final_df['Text'][2]

"It can't be the screen then. Definitely something out there, Captain, headed this way."

In [ ]:
final_df

,Episode,Scene,Character,Text
0,1,Bridge,SPOCK,Check the circuit.
1,1,Bridge,TYLER,"All operating, sir."
2,1,Bridge,SPOCK,It can't be the screen then. Definitely someth...
3,1,Bridge,TYLER,It could be these meteorites.
4,1,Bridge,ONE,"No, it's something else. There's still somethi..."
...,...,...,...,...
40280,None,Ba'ku village hayfield,DATA,Bye.
40281,None,Ba'ku village hayfield,SOJEF,"Mister Data, I hope we'll see you again."
40282,None,Ba'ku village hayfield,ARTIM,Data! ...Don't forget. You have to have a litt...
40283,None,Ba'ku village hayfield,RIKER,Good advice.


In [ ]:
import pandas as pd

def process_spock_dialogue(df, N=7):
    processed_data = []

    # Group by episodes and scenes
    grouped = df.groupby(["Episode", "Scene"])

    for (episode, scene), scene_data in grouped:
        scene_data = scene_data.reset_index(drop=True)

        for i, row in scene_data.iterrows():
            if row["Character"].lower() != "spock":
                continue  # Only process Spock's lines

            answer = row["Text"]

            # Find the preceding utterance (QUESTION)
            if i > 0:
                question = scene_data.iloc[i - 1]["Text"]
            else:
                question = ""  # No preceding utterance

            # Build CONTEXT progressively
            for n in range(1, N + 1):
                if i - n - 1 < 0:
                    break  # Stop if out of bounds

                context_list = scene_data.iloc[max(0, i - n - 1):i - 1]["Text"].tolist()
                context = " ".join(context_list) if context_list else ""

                # Append QUESTION and ANSWER to CONTEXT
                full_context = f"{context} [SEP] {question} [SEP] {answer}".strip()

                processed_data.append({
                    "QUESTION": question,
                    "ANSWER": answer,
                    "CONTEXT": context,
                  #  "CONTEXT": full_context,
                  #  "Episode": episode,
                  #  "Scene": scene
                })

    return pd.DataFrame(processed_data)

processed_df = pd.DataFrame()

processed_df = process_spock_dialogue(final_df, N=10)


In [ ]:
processed_df.shape

(38024, 3)

In [ ]:
processed_df[:10]

,QUESTION,ANSWER,CONTEXT
0,"All operating, sir.",It can't be the screen then. Definitely someth...,"Check the circuit. ['SEP'] All operating, sir...."
1,We've no ships or Earth colonies that far out.,Their call letters check with a survey expedit...,I have a fix. It comes from the Talos star gro...
2,We've no ships or Earth colonies that far out.,Their call letters check with a survey expedit...,"A ship in trouble making a forced landing, sir..."
3,We've no ships or Earth colonies that far out.,Their call letters check with a survey expedit...,They were keyed to cause interference and attr...
4,We've no ships or Earth colonies that far out.,Their call letters check with a survey expedit...,"It's a radio wave, sir. We're passing through ..."
5,We've no ships or Earth colonies that far out.,Their call letters check with a survey expedit...,"Steady as we go. It's a radio wave, sir. We're..."
6,We've no ships or Earth colonies that far out.,Their call letters check with a survey expedit...,"Evasive manoeuvres, sir? Steady as we go. It's..."
7,We've no ships or Earth colonies that far out.,Their call letters check with a survey expedit...,"It's coming at the speed of light, collision c..."
8,We've no ships or Earth colonies that far out.,Their call letters check with a survey expedit...,"No, it's something else. There's still somethi..."
9,We've no ships or Earth colonies that far out.,Their call letters check with a survey expedit...,"It could be these meteorites. No, it's somethi..."


In [ ]:
processed_df['CONTEXT'][5]

"Steady as we go. It's a radio wave, sir. We're passing through an old-style distress signal. They were keyed to cause interference and attract attention this way. A ship in trouble making a forced landing, sir. That's it. No other message. I have a fix. It comes from the Talos star group. [SEP] We've no ships or Earth colonies that far out. [SEP] Their call letters check with a survey expedition. SS Columbia. It disappeared in that region approximately eighteen years ago."

In [ ]:
processed_df.to_pickle(data_path+"spock_lines_context.pkl")